In [18]:
import numpy as np
import math
import sys

def read_data():
    x = []
    y = []
    with open('iris.data', 'r') as f:
        for line in f:
            if line == '\n':
                continue
            sepal_l, sepal_w, petal_l, petal_w, iris_class = line.split(',')
            x.append([1, sepal_l, sepal_w, petal_l, petal_w])
            if iris_class == 'Iris-setosa\n':
                y.append(1)
            elif iris_class == 'Iris-versicolor\n':
                y.append(2)
            else:
                y.append(3)
    return np.array(x).astype('float64'), np.array(y).astype('float64')

def logistic_func(x, w):
    return float(1) / (1 + math.e**(-x.dot(w)))

def cost_func(x, y, reg, w):
    log_func_v = logistic_func(x, w)
    y = np.squeeze(y)
    step1 = y * np.log(log_func_v)
    step2 = (1 - y) * np.log(1 - log_func_v)
    final = -step1 - step2
    return np.mean(final) + reg * w.dot(w)

def sgd(reg, lr, T, x, y):
    w = np.zeros(x.shape[1]);
    
    sum_w = w
    for t in xrange(0, T): 
        idx = np.random.randint(x.shape[0])
        w = w - lr * ((logistic_func(x[idx], w) - y[idx]) * x[idx] + 2 * reg * w)
        sum_w = sum_w + w
    return sum_w / T

def bgd(reg, lr, T, x, y):
    w = np.zeros(x.shape[1]);
    
    for t in xrange(0, T): 
        sum_it = np.zeros(x.shape[1]);
        for idx in xrange(0, x.shape[0]):
            sum_it = sum_it + lr * ((logistic_func(x[idx], w) - y[idx]) * x[idx] + 2 * reg * w)
        w = w - sum_it / x.shape[0]
    return w

def train(iris_class, x, y, reg, lr, T, alg):
    y[y != iris_class] = 0
    y[y == iris_class] = 1

    return alg(reg, lr, T, x, y)
    
def validation(iris_class, w, x, y, reg):
    y[y != iris_class] = 0
    y[y == iris_class] = 1
    
    return cost_func(x, y, reg, w)
    
def cross_validation(k, iris_class, x, y, alg):
    best_err = sys.maxint
    best_reg = 0
    best_lr = 0
    best_T = 0
    
    for reg in [0.001, 0.005, 0.01, 0.1, 0.3, 0.5]:
        for lr in [0.001, 0.005, 0.01, 0.1, 0.3, 0.5]:
            for T in [20, 40, 60, 100, 200, 300]:
                res = 0.0
                for k_idx in xrange(0, k):                    
                    x_train = np.array([item for i, item in enumerate(x) if i % k != k_idx])
                    y_train = np.array([item for i, item in enumerate(y) if i % k != k_idx])
                    
                    x_validation = np.array([item for i, item in enumerate(x) if i % k == k_idx])
                    y_validation = np.array([item for i, item in enumerate(y) if i % k == k_idx])
                    
                    w = train(iris_class, x_train, y_train, reg, lr, T, alg)
                    res = res + validation(iris_class, w, x_validation, y_validation, reg)
                res = res / k
                if res < best_err:
                    best_err = res
                    best_reg = reg
                    best_lr = lr
                    best_T = T
    return best_err, best_reg, best_lr, best_T 
    

x, y = read_data()

c = np.c_[x.reshape(len(x), -1), y.reshape(len(y), -1)]
np.random.shuffle(c)

x_train = c[:, :x.size//len(x)][:int(0.9 * x.shape[0])]
y_train = c[:, x.size//len(x):][:int(0.9 * x.shape[0])]

x_test = c[:, :x.size//len(x)][int(0.9 * x.shape[0]):]
y_test = c[:, x.size//len(x):][int(0.9 * x.shape[0]):]

for i in xrange(1, 4):
    print str(i) + " SGD "
    err, reg, lr, T = cross_validation(5, i, x_train.copy(), y_train.copy(), sgd)        
    print err, reg, lr, T
    w = train(i, x_train.copy(), y_train.copy(), reg, lr, T, sgd)
    print validation(i, w, x_test.copy(), y_test.copy(), reg)
    
    print str(i) + " BGD "
    err, reg, lr, T = cross_validation(5, i, x_train.copy(), y_train.copy(), bgd)
    print err, reg, lr, T
    w = train(i, x_train.copy(), y_train.copy(), reg, lr, T, bgd)
    print validation(i, w, x_test.copy(), y_test.copy(), reg)

1 SGD 
0.0232365190735 0.001 0.3 300
0.0230350216217
1 BGD 
0.0212252944103 0.001 0.5 300
0.0202722298378
2 SGD 
0.615848932201 0.001 0.01 300
0.568829754682
2 BGD 
0.56260034151 0.001 0.1 300
0.485275538614
3 SGD 
0.182480920307 0.001 0.3 300
0.0947466704998
3 BGD 
0.154970444271 0.001 0.5 300
0.142387432615
